# Project No. 3 - Bouquet of Flowers
### Authors:
M. Alejandro Villalobos C.
Óscar Ruiz Ramirez
Sofía Vargas Aceves
### Fecha:
24 de Marzo, 2022
### Description:
Tercer proyecto Machine Learning. Bouquet of Flowers Classification

## 1. DATA READ


In [1]:
import pandas as pd

Dataset = pd.read_csv("./Embedded_images.csv")

Dataset.keys()

Index(['n0', 'n1', 'n2', 'n3', 'n4', 'n5', 'n6', 'n7', 'n8', 'n9',
       ...
       'n2044', 'n2045', 'n2046', 'n2047', 'category', 'image name', 'image',
       'size', 'width', 'height'],
      dtype='object', length=2054)

In [2]:
Dataset = Dataset.drop(['image','image name', 'size', 'width','height'],axis=1)
DataFrameFlowers = pd.DataFrame(Dataset)

## 2. Data Preprocessing

In [120]:
X = DataFrameFlowers.drop(['category'], axis = 1)
Y = DataFrameFlowers['category']


from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.20, random_state=0)

## Model #1 Logistic Regression

### 3. Model Creation

In [59]:
# Implementado con SAGA por el tamaño de muestra, referenciando (6)
from sklearn.linear_model import LogisticRegression
LR = LogisticRegression(C=50.0 / y_train.shape[0], penalty="l1", solver="saga", tol=0.1)

### 4. Training: Adjust Model with Historic Data

In [60]:
LR.fit(X_train, y_train)

LogisticRegression(C=0.24875621890547264, penalty='l1', solver='saga', tol=0.1)

### 5. Prediction for New Data

In [61]:
y_predictLR = LR.predict(X_test)

## Model #2 Convolutional Neural Network (CNN)

In [121]:
#Normalizando imagen de [0,1] para el modelo de CNN
#X_trainCNN = X_train/255.0
#X_testCNN = X_test/255.0

#Aqui haremos las variables especialmente para el modelo CNN, ya que se necesita estar en 3 dimensiones, 28 x 28 x 1
#Código complementado con (5)
X_trainCNN = X_train.values.reshape(201,2048,1)
X_testCNN = X_test.values.reshape(51,2048,1)

In [114]:
X_train.shape

(126, 2048)

### 3. Model Creation

In [128]:
import numpy as np
import keras 
from keras.models import Sequential
from keras.layers import  Dense, Conv1D, Flatten, MaxPooling1D

#Para coincidir tamaños, se convierten las etiquetas en "one hot vectors"
#Código complementado con (5)
y_trainCNN = keras.utils.to_categorical(np.asarray(y_train.factorize()[0]))
#y_trainCNN = keras.utils.to_categorical(y_train)
#y_testCNN = keras.utils.to_categorical(y_test)
y_testCNN = keras.utils.to_categorical(np.asarray(y_test.factorize()[0]))

CNN = Sequential()
CNN.add(Conv1D(128, 3, activation='relu', input_shape=(2048,1)))
CNN.add(MaxPooling1D((2)))
CNN.add(Conv1D(256, 3, activation='relu'))
CNN.add(MaxPooling1D((2)))
CNN.add(Conv1D(256, 3, activation='relu'))
CNN.add(MaxPooling1D((2)))

CNN.add(Flatten())
CNN.add(Dense(256, activation='relu'))

#Capa Dense tamaño 10 por las 10 categorizaciones finales
#Complementado con activación softmax según (7)
CNN.add(Dense(10, activation='softmax'))

#Complementado con loss según (7)
CNN.compile(optimizer='adam', loss = keras.losses.categorical_crossentropy, metrics=['accuracy'])

In [27]:
y_testCNN.shape

(51, 10)

In [ ]:
X

In [28]:
y_trainCNN.shape

(201, 10)

### 4. Training: Adjust Model with Historic Data

In [129]:
CNN_H = CNN.fit(X_trainCNN, y_trainCNN, validation_data=(X_testCNN, y_testCNN), epochs=10)

Epoch 1/10
7/7 [==============================] - 6s 833ms/step - loss: 2.7450 - accuracy: 0.0995 - val_loss: 2.2721 - val_accuracy: 0.3333
Epoch 2/10
7/7 [==============================] - 5s 689ms/step - loss: 2.2512 - accuracy: 0.2090 - val_loss: 2.2962 - val_accuracy: 0.1176
Epoch 3/10
7/7 [==============================] - 4s 632ms/step - loss: 2.0385 - accuracy: 0.4080 - val_loss: 2.1623 - val_accuracy: 0.3725
Epoch 4/10
7/7 [==============================] - 5s 706ms/step - loss: 1.4400 - accuracy: 0.5721 - val_loss: 2.3979 - val_accuracy: 0.2549
Epoch 5/10
7/7 [==============================] - 5s 736ms/step - loss: 0.9900 - accuracy: 0.6866 - val_loss: 2.7852 - val_accuracy: 0.3529
Epoch 6/10
7/7 [==============================] - 5s 687ms/step - loss: 0.5670 - accuracy: 0.8209 - val_loss: 3.5782 - val_accuracy: 0.3725
Epoch 7/10
7/7 [==============================] - 5s 770ms/step - loss: 0.3654 - accuracy: 0.8806 - val_loss: 4.8307 - val_accuracy: 0.3725
Epoch 8/10
7/7 [====

### 5. Prediction for New Data

In [ ]:
y_predictCNN = CNN.predict_classes(X_testCNN)

## Model #3 Support Vector Machine (SVM)

### 3. Model Creation

In [31]:
from sklearn import svm
SVM = svm.SVC()

### 4. Training: Adjust Model with Historic Data

In [32]:
SVM.fit(X_train, y_train)

SVC()

### 5. Prediction for New Data

In [33]:
y_predictSVM = SVM.predict(X_test)

## Model #4 Random Forest

### 3. Model Creation

In [161]:
from sklearn.ensemble import RandomForestClassifier
RFC = RandomForestClassifier(n_estimators=500)

### 4. Training: Adjust Model with Historic Data

In [162]:
RFC.fit(X_train, y_train)

RandomForestClassifier(n_estimators=500)

### 5. Prediction for New Data

In [163]:
y_predictRF = RFC.predict(X_test)

## 6. Visualization of Results

In [164]:
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score

### Model #1 Logistic Regression

In [38]:
CV_LR = cross_val_score(LR, X_train, y_train, cv=3, scoring = "accuracy")

In [39]:
AS_LR = accuracy_score(y_test,y_predictLR)

### Model #2 Support Vector Machine (SVM)

In [40]:
CV_SVM = cross_val_score(SVM, X_train, y_train, cv=3, scoring = "accuracy")

In [41]:
AS_SVM = accuracy_score(y_test,y_predictSVM)

### Model #3 Random Forest

In [168]:
CV_RF = cross_val_score(RFC, X_train, y_train, cv=3, scoring = "accuracy")

In [169]:
AS_RF = accuracy_score(y_test,y_predictRF)

### Model #5 Convolutional Neural Network (CNN)

In [130]:
CV_CNN = CNN.evaluate(X_trainCNN, y_trainCNN)

7/7 [==============================] - 1s 184ms/step - loss: 0.0581 - accuracy: 0.9851


In [131]:
AS_CNN = CNN.evaluate(X_testCNN, y_testCNN)

2/2 [==============================] - 0s 63ms/step - loss: 5.3975 - accuracy: 0.3922


In [170]:
#Creación de dataframe con los resultados
models = ["LR","SVM", "RF", "CNN"]
AS_values = [AS_LR, AS_SVM, AS_RF,  AS_CNN[1]]
CV_values = [CV_LR, CV_SVM, CV_RF, [CV_CNN[1],'-','-']]

data = {"Accuracy Scores": AS_values, "Cross Validations": CV_values}
df = pd.DataFrame(data, index = models)

#Separación de arrays de Cross Validations en columnas, referenciando (10)
dfCV = pd.DataFrame(df['Cross Validations'].to_list(), columns=['CV #1','CV #2','CV #3'], index = models)
#Cálculo de promedio de Cross Validations
dfCV.insert(0,'CV AVG', dfCV[['CV #1','CV #2','CV #3']].mean(axis=1, numeric_only=True))

#Concatenación de dataframes con las columnas finales, referenciando (11)
dfAcc = pd.concat([df['Accuracy Scores'], dfCV], axis=1)
dfAcc = dfAcc.style.set_caption("Models' Accuracy Scores and Cross Validations")
dfAcc

,Accuracy Scores,CV AVG,CV #1,CV #2,CV #3
LR,0.686275,0.626866,0.626866,0.641791,0.731343
SVM,0.647059,0.641791,0.641791,0.716418,0.686567
RF,0.725490,0.686567,0.686567,0.731343,0.686567
CNN,0.392157,0.985075,0.985075,-,-
